In [1]:
import os, time, sys
from vllm import LLM
sys.path.append('../elyra_pipeline/')
from postgres_utilities import connect_db, close_db, execute_sql_results, execute_sql_results_np

INFO 05-05 13:37:06 [__init__.py:239] Automatically detected platform cuda.


In [2]:
model_name = "GritLM/GritLM-7B"
number_data_rows = 1000

In [3]:
try:
    conn = connect_db()
    sql = f"select productdisplayname from products limit {number_data_rows}"
    results = execute_sql_results_np(conn, sql)
except Exception as e:
    raise e
finally:
    close_db(conn)

In [4]:
llm = LLM(model=model_name, task="embed", max_model_len=4096, quantization="fp8")

WARNING 05-05 13:37:18 [arg_utils.py:1658] --task embed is not supported by the V1 Engine. Falling back to V0. 
INFO 05-05 13:37:19 [llm_engine.py:240] Initializing a V0 LLM engine (v0.8.5.post1) with config: model='GritLM/GritLM-7B', speculative_config=None, tokenizer='GritLM/GritLM-7B', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=4096, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=fp8, enforce_eager=False, kv_cache_dtype=auto,  device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='auto', reasoning_backend=None), observability_config=ObservabilityConfig(show_hidden_metrics=False, otlp_traces_endpoint=None, collect_model_forward_time=False, collect_model_execute_time=False), seed=None, served_model_name=GritLM/GritLM-7B, num_scheduler_ste

Loading safetensors checkpoint shards:   0% Completed | 0/3 [00:00<?, ?it/s]


INFO 05-05 13:37:22 [loader.py:458] Loading weights took 2.54 seconds
WARNING 05-05 13:37:22 [marlin_utils_fp8.py:54] Your GPU does not have native support for FP8 computation but FP8 quantization is being used. Weight-only FP8 compression will be used leveraging the Marlin kernel. This may degrade performance for compute-heavy workloads.
WARNING 05-05 13:37:22 [kv_cache.py:128] Using Q scale 1.0 and prob scale 1.0 with fp8 attention. This may cause accuracy issues. Please make sure Q/prob scaling factors are available in the fp8 checkpoint.
INFO 05-05 13:37:23 [model_runner.py:1140] Model loading took 6.7564 GiB and 3.460481 seconds


In [5]:
test_description = "Starting batch embedding test using local vllm"
print(test_description)
print(len(test_description) * "-") 
tic = time.perf_counter()
print("Embeddings computed ...", end=" ")
descriptions = list()
for index, product in enumerate(results):
    descriptions.append(product[0])

batch_size = 1000
start_offset = 0
end_offset = batch_size
embeddings = list()
while start_offset < number_data_rows:
    print(start_offset, end_offset, end=" ")
    embeddings = embeddings + llm.embed(descriptions[start_offset:end_offset])
    start_offset = end_offset
    end_offset = end_offset + batch_size
    if end_offset > number_data_rows:
        end_offset = number_data_rows

assert len(embeddings) == index+1
for e in embeddings:
    assert(e.outputs.hidden_size == 4096)

toc = time.perf_counter()
print("")
print(f"Time to compute {number_data_rows} embeddings: {toc - tic:0.4f} seconds")
print(len(test_description) * "-") 

Starting batch embedding test using local vllm
----------------------------------------------
Embeddings computed ... 0 1000 

Processed prompts:   0%|          | 0/1000 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s…


Time to compute 1000 embeddings: 3.7761 seconds
----------------------------------------------
